In [27]:
!pip install torch wandb gymnasium numpy gymnasium[other] gymnasium[classic-control]


  Using cached pygame-2.6.1-cp313-cp313-win_amd64.whl.metadata (13 kB)
Using cached pygame-2.6.1-cp313-cp313-win_amd64.whl (10.6 MB)


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import wandb
import os
import random
import math
import gymnasium as gym
import numpy as np
from collections import deque, namedtuple
import time
from gymnasium.wrappers import RecordVideo
import itertools

In [ ]:
wandb.login(key="")
wandb.init(project="RL assignment 2", name="assignment2_run1")

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\medor\_netrc
wandb: Currently logged in as: omarelshereef (omarelshereef-cairo-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [3]:
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))

class ReplayMemory(object):
    """
    A simple replay buffer, as described in the DQN paper and lecture.
    Stores transitions and allows for random sampling of batches.
    """
    def __init__(self, capacity):
        # Use a deque as the memory. It automatically handles max length.
        self.memory = deque([], maxlen=capacity)

    def push(self, *args):
        """Save a transition (state, action, next_state, reward)"""
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        """Select a random batch of transitions for training"""
        return random.sample(self.memory, batch_size)

    def __len__(self):
        """Return the current size of the memory"""
        return len(self.memory)

In [4]:
class DQN(nn.Module):
    """
    The Deep Q-Network model.
    It's a simple feed-forward neural network.
    Input: State (s)
    Output: Q-value for each possible action Q(s, a)
    """
    def __init__(self, n_observations, n_actions):
        super(DQN, self).__init__()
        # Define the layers, matching the lecture slide
        self.layer1 = nn.Linear(n_observations, 128)
        self.layer2 = nn.Linear(128, 128)
        self.layer3 = nn.Linear(128, n_actions)

    def forward(self, x):
        """
        Defines the forward pass of the network.
        Uses ReLU activation functions as shown in the lecture
        """
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        # The final layer returns raw Q-values (no activation)
        return self.layer3(x)

In [5]:
# Use the GPU if available, otherwise use the CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def select_action(state, policy_net, n_actions, epsilon):
    """
    Selects an action using an epsilon-greedy policy.
    With probability (1-epsilon), it exploits (picks the best action).
    With probability (epsilon), it explores (picks a random action).
    """
    sample = random.random()
    if sample > epsilon:
        # EXPLOITATION: Get the best action from the policy_net
        with torch.no_grad():
            # policy_net(state) returns Q-values for all actions
            # .max(1)[1] gets the *index* of the max Q-value
            # .view(1, 1) reshapes it to [[action]]
            return policy_net(state).max(1)[1].view(1, 1)
    else:
        # EXPLORATION: Pick a random action
        return torch.tensor([[random.randrange(n_actions)]], device=device, dtype=torch.long)

In [6]:
class DiscretizeActionWrapper(gym.ActionWrapper):
    """
    Wraps the Pendulum-v1 environment to discretize its continuous action space.
    The continuous action is a torque in [-2.0, 2.0].
    We will map 3 discrete actions {0, 1, 2} to {-2.0, 0.0, 2.0}.
    """
    def __init__(self, env, num_discrete_actions=3):
        super().__init__(env)
        self.num_actions = num_discrete_actions
        # Redefine the action space as discrete
        self.action_space = gym.spaces.Discrete(self.num_actions)
        # Create a mapping from discrete action index to continuous torque value
        self.action_map = np.linspace(
            self.env.action_space.low[0],
            self.env.action_space.high[0],
            self.num_actions
        )

    def action(self, action_index):
        # Map the discrete action index back to a continuous value
        continuous_action = [self.action_map[action_index]]
        return np.array(continuous_action, dtype=np.float32)

In [7]:
def optimize_model(policy_net, target_net, optimizer, memory, batch_size, gamma, loss_fn):
    """
    Performs one step of DQN optimization (backpropagation).
    Samples a batch, computes the loss, and updates the policy_net.
    """
    if len(memory) < batch_size:
        return  # Not enough experiences in memory to sample a batch

    # Sample a batch of transitions from replay memory
    transitions = memory.sample(batch_size)
    batch = Transition(*zip(*transitions))

    # Create tensors for states, actions, and rewards
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)

    # Identify non-final next states
    non_final_mask = torch.tensor(tuple(s is not None for s in batch.next_state), device=device, dtype=torch.bool)
    non_final_next_states = torch.cat([s for s in batch.next_state if s is not None])

    # Compute Q_current: Q(s, a) for the actions taken
    Q_current = policy_net(state_batch).gather(1, action_batch)

    # Compute Q_target: R + gamma * max_a' Q(s', a')
    Q_target_next = torch.zeros(batch_size, device=device)
    with torch.no_grad():
        # DQN: Get max Q-value from target network
        Q_target_next[non_final_mask] = target_net(non_final_next_states).max(1)[0]

    Q_target = reward_batch + (Q_target_next * gamma)

    # Compute loss (Smooth L1 / Huber Loss)
    loss = loss_fn(Q_current, Q_target.unsqueeze(1))

    # Backpropagation
    optimizer.zero_grad()
    loss.backward()
    torch.nn.utils.clip_grad_value_(policy_net.parameters(), 100)
    optimizer.step()

In [8]:
# --- Main Training Function ---
def run_experiment(
    env_name,
    num_episodes=500,
    batch_size=128,
    gamma=0.99,
    eps_decay=1000,
    tau=0.005,
    lr=1e-4,
    memory_size=10000,
    wandb_run_name=None,
    wandb_group=None
):
    eps_start = 0.9
    eps_end = 0.05

    # Initialize wandb
    if wandb_run_name is None:
        wandb_run_name = f"{env_name}_DQN_{int(time.time())}"

    config = {
        "env_name": env_name,
        "model_type": "DQN",
        "num_episodes": num_episodes,
        "batch_size": batch_size,
        "gamma": gamma,
        "eps_start": eps_start,
        "eps_end": eps_end,
        "eps_decay": eps_decay,
        "tau": tau,
        "lr": lr,
        "memory_size": memory_size,
    }

    run = wandb.init(
        project="cmps458_assignment2",
        name=wandb_run_name,
        group=wandb_group,
        config=config
    )

    # Setup Environment
    if env_name == "Pendulum-v1":
        env = gym.make(env_name)
        env = DiscretizeActionWrapper(env, num_discrete_actions=5)
    else:
        env = gym.make(env_name)

    n_actions = env.action_space.n
    state, info = env.reset()
    n_observations = len(state)

    # Initialize Networks and Optimizer
    policy_net = DQN(n_observations, n_actions).to(device)
    target_net = DQN(n_observations, n_actions).to(device)
    target_net.load_state_dict(policy_net.state_dict())
    target_net.eval()

    optimizer = torch.optim.AdamW(policy_net.parameters(), lr=lr, amsgrad=True)
    memory = ReplayMemory(memory_size)
    loss_fn = nn.SmoothL1Loss()

    print(f"--- Starting Training: {wandb_run_name} (Group: {wandb_group}) ---")

    # Training Loop
    steps_done = 0
    for i_episode in range(num_episodes):
        state, info = env.reset()
        state = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)

        episode_duration = 0
        episode_reward = 0
        
        for t in itertools.count():
            epsilon = eps_end + (eps_start - eps_end) * math.exp(-1. * steps_done / eps_decay)
            action = select_action(state, policy_net, n_actions, epsilon)
            steps_done += 1

            observation, reward, terminated, truncated, _ = env.step(action.item())
            episode_duration += 1
            episode_reward += reward
            reward = torch.tensor([reward], device=device)
            done = terminated or truncated

            next_state = None if terminated else torch.tensor(observation, dtype=torch.float32, device=device).unsqueeze(0)
            memory.push(state, action, next_state, reward)
            state = next_state

            # Optimize model
            optimize_model(policy_net, target_net, optimizer, memory, batch_size, gamma, loss_fn)

            # Soft update of target network (more efficient)
            with torch.no_grad():
                for target_param, policy_param in zip(target_net.parameters(), policy_net.parameters()):
                    target_param.data.mul_(1 - tau).add_(policy_param.data, alpha=tau)

            if done:
                break

        # Log episode metrics
        wandb.log({
            "episode": i_episode,
            "duration": episode_duration,
            "episode_reward": episode_reward,
            "epsilon": epsilon
        })

    print(f"--- Training Complete: {wandb_run_name} ---")
    model_path = f"{wandb_run_name}.pth"
    torch.save(policy_net.state_dict(), model_path)
    run.finish()
    return model_path, config


# --- Test & Record Function ---
def test_and_record(
    env_name,
    model_type,
    model_path,
    num_tests=100,
    wandb_run_name=None,
    wandb_group=None,
    config=None
):
    print(f"\n--- Testing {model_type} on {env_name} ---")

    if wandb_run_name is None:
        wandb_run_name = f"test_{model_type}"

    if config is None:
        config = {"env_name": env_name, "model_type": model_type, "num_tests": num_tests}

    run = wandb.init(
        project="cmps458_assignment2_tests",
        name=wandb_run_name,
        group=wandb_group,
        config=config,
        reinit=True
    )

    # Setup Environment
    base_env = gym.make(env_name, render_mode="rgb_array")
    env = RecordVideo(
        base_env,
        video_folder=f"videos/{wandb_run_name}_{int(time.time())}",
        episode_trigger=lambda e: e == 0,
        name_prefix=f"{model_type}",
        disable_logger=True
    )

    # Load Model
    state, info = env.reset()
    env.render()  # ✅ Initialize render buffer once here

    n_actions = env.action_space.n
    n_observations = len(state)
    model = DQN(n_observations, n_actions).to(device)
    model.load_state_dict(torch.load(model_path, map_location=device, weights_only=True))
    model.eval()

    test_durations, test_rewards = [], []
    for i_episode in range(num_tests):
        state, info = env.reset()
        env.render()  # ✅ Ensure rendering is active for each episode
        state = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)

        episode_duration = 0
        episode_reward = 0

        for t in itertools.count():
            with torch.no_grad():
                action = select_action(state, model, n_actions, epsilon=0.0)
            observation, reward, terminated, truncated, _ = env.step(action.item())
            env.render()  # ✅ Capture frame

            episode_duration += 1
            episode_reward += reward

            if terminated or truncated:
                break

            state = torch.tensor(observation, dtype=torch.float32, device=device).unsqueeze(0)

        wandb.log({"test_episode": i_episode, "test_duration": episode_duration, "test_reward": episode_reward})
        test_durations.append(episode_duration)
        test_rewards.append(episode_reward)

    env.close()  # ✅ Properly close to finalize video

    video_folder = env.video_folder
    video_files = [f for f in os.listdir(video_folder) if f.endswith('.mp4')]
    if video_files:
        video_path = os.path.join(video_folder, video_files[0])
        print(f"Video file: {video_path}")
        wandb.log({"test_video": wandb.Video(video_path, fps=30, format="mp4")})

    avg_duration = sum(test_durations) / num_tests
    avg_reward = sum(test_rewards) / num_tests
    print(f"\nTesting complete. Video saved in '{video_folder}'")
    print(f"Average duration: {avg_duration:.2f}, Average reward: {avg_reward:.2f}")
    run.finish()

In [ ]:
# --- acrobot-v1: DQN ---
group_name = "Acrobot_test"
run_name = f"Acrobot_test"
model_path, config = run_experiment(
    env_name="Acrobot-v1",
    num_episodes=500,
    wandb_run_name=run_name,
    wandb_group=group_name
)
test_and_record(
    "Acrobot-v1", run_name, model_path,
    wandb_run_name=f"test_{run_name}",
    wandb_group=group_name,
    config=config
)


wandb: ERROR The nbformat package was not found. It is required to save notebook history.


--- Starting Training for CartPole_DQN_lr-0.001 (Group: CartPole_LR_Test) ---
Config: {'env_name': 'CartPole-v1', 'model_type': 'DQN', 'num_episodes': 500, 'batch_size': 128, 'gamma': 0.99, 'eps_start': 0.9, 'eps_end': 0.05, 'eps_decay': 1000, 'tau': 0.005, 'lr': 0.0001, 'memory_size': 10000}


KeyboardInterrupt: 

In [ ]:
# --- Acrobot-v1 Hyperparameter Sweep ---

# 1. Test Learning Rates (lr)
lr_tests = [1e-3, 1e-4]
for lr in lr_tests:
    group_name = "Acrobot_LR_Test"
    run_name = f"Acrobot_DQN_lr-{lr}"

    model_path, config = run_experiment(
        env_name="Acrobot-v1",
        num_episodes=500,
        lr=lr,  # <-- Variable we are testing
        wandb_run_name=run_name,
        wandb_group=group_name
    )
    test_and_record(
        "Acrobot-v1", run_name, model_path,
        wandb_run_name=f"test_{run_name}",
        wandb_group=group_name,
        config=config
    )

# 2. Test Discount Factors (gamma)
gamma_tests = [0.99, 0.9]
for gamma in gamma_tests:
    group_name = "Acrobot_Gamma_Test"
    run_name = f"Acrobot_DQN_gamma-{gamma}"

    model_path, config = run_experiment(
        env_name="Acrobot-v1",
        num_episodes=500,
        gamma=gamma,  # <-- Variable we are testing
        wandb_run_name=run_name,
        wandb_group=group_name
    )
    test_and_record(
        "Acrobot-v1", run_name, model_path,
        wandb_run_name=f"test_{run_name}",
        wandb_group=group_name,
        config=config
    )

# 1. Test Epsilon Decay Rates
eps_decay_tests = [1000, 5000]
for eps_decay in eps_decay_tests:
    group_name = "Acrobot_EpsDecay_Test"
    run_name = f"Acrobot_DDQN_eps-{eps_decay}" # Let's use DDQN, it's more stable

    model_path, config = run_experiment(
        env_name="Acrobot-v1", num_episodes=1000, lr=1e-3,
        eps_decay=eps_decay, # <-- Variable we are testing
        wandb_run_name=run_name, wandb_group=group_name
    )
    test_and_record(
        "Acrobot-v1", run_name, model_path,
        wandb_run_name=f"test_{run_name}", wandb_group=group_name, config=config
    )

# 2. Test Replay Memory Sizes
memory_tests = [10000, 50000]
for mem_size in memory_tests:
    group_name = "Acrobot_Memory_Test"
    run_name = f"Acrobot_DDQN_mem-{mem_size}"

    model_path, config = run_experiment(
        env_name="Acrobot-v1", num_episodes=1000, lr=1e-3,
        memory_size=mem_size, # <-- Variable we are testing
        wandb_run_name=run_name, wandb_group=group_name
    )
    test_and_record(
        "Acrobot-v1", run_name, model_path,
        wandb_run_name=f"test_{run_name}", wandb_group=group_name, config=config
    )

In [ ]:
def record_model(model_path, env_name, num_discrete_actions=5):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # --- Setup environment ---
    base_env = gym.make(env_name, render_mode="rgb_array")

    # If continuous, apply discretizer
    if isinstance(base_env.action_space, gym.spaces.Box):
        env = DiscretizeActionWrapper(base_env, num_discrete_actions=num_discrete_actions)
    else:
        env = base_env

    env = RecordVideo(env, video_folder=f"videos/{env_name}", episode_trigger=lambda e: True)

    # --- Reset and get sizes ---
    state, _ = env.reset()
    env.render()  # important to initialize frame buffer

    n_actions = env.action_space.n
    n_observations = len(state)

    # --- Load model ---
    model = DQN(n_observations, n_actions).to(device)
    model.load_state_dict(torch.load(model_path, map_location=device, weights_only=True))
    model.eval()

    # --- Run one episode ---
    state = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)
    total_reward, done = 0, False

    while not done:
        with torch.no_grad():
            q_values = model(state)
            action = torch.argmax(q_values).item()

        next_state, reward, terminated, truncated, _ = env.step(action)
        env.render()
        total_reward += reward
        done = terminated or truncated
        state = torch.tensor(next_state, dtype=torch.float32, device=device).unsqueeze(0)

    env.close()
    print(f"✅ Video saved in ./videos/{env_name}/ (total reward = {total_reward:.2f})")


In [66]:
record_model("./Pendulum_DQN_eps-1000.pth", "Pendulum-v1")

d:\materials for college\Fall 2025-2026\Reinforcement Learning\assignments\assignment 2\.venv\Lib\site-packages\gymnasium\wrappers\rendering.py:293: UserWarning: WARN: Overwriting existing videos at d:\materials for college\Fall 2025-2026\Reinforcement Learning\assignments\assignment 2\videos\Pendulum-v1 folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


✅ Video saved in ./videos/Pendulum-v1/ (total reward = -124.18)


d:\materials for college\Fall 2025-2026\Reinforcement Learning\assignments\assignment 2\.venv\Lib\site-packages\gymnasium\wrappers\rendering.py:434: UserWarning: WARN: Unable to save last video! Did you call close()?
  logger.warn("Unable to save last video! Did you call close()?")


In [9]:
def record_models_in_folder(models_folder, env_name, num_discrete_actions=5, num_tests=100):
    """
    Loads all DQN models in a folder, runs multiple test episodes per model,
    and records one random episode per model as a video.
    """

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # --- Get all model paths ---
    model_files = [f for f in os.listdir(models_folder) if f.endswith(".pth")]
    if not model_files:
        print("⚠️ No .pth models found in the folder!")
        return

    for model_file in model_files:
        model_path = os.path.join(models_folder, model_file)
        print(f"\n🎬 Testing and recording model: {model_file}")

        # --- Create environment ---
        base_env = gym.make(env_name, render_mode="rgb_array")

        # Apply discretization if needed
        if isinstance(base_env.action_space, gym.spaces.Box):
            env = DiscretizeActionWrapper(base_env, num_discrete_actions=num_discrete_actions)
        else:
            env = base_env

        # --- Setup model ---
        sample_state, _ = env.reset()
        n_observations = len(sample_state)
        n_actions = env.action_space.n

        model = DQN(n_observations, n_actions).to(device)
        model.load_state_dict(torch.load(model_path, map_location=device))
        model.eval()

        # --- Run test episodes ---
        rewards = []
        episodes_data = []

        for ep in range(num_tests):
            state, _ = env.reset()
            total_reward = 0
            done = False

            episode_states, episode_actions = [], []

            while not done:
                state_tensor = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)
                with torch.no_grad():
                    q_values = model(state_tensor)
                    action = torch.argmax(q_values).item()

                next_state, reward, terminated, truncated, _ = env.step(action)
                total_reward += reward
                done = terminated or truncated
                state = next_state

                # Save episode trajectory for later if we want to record this one
                episode_states.append(state)
                episode_actions.append(action)

            rewards.append(total_reward)
            episodes_data.append((episode_states, episode_actions))

            print(f"  Episode {ep+1}/{num_tests}: reward = {total_reward:.2f}")

        avg_reward = np.mean(rewards)
        print(f"✅ Model {model_file}: avg reward = {avg_reward:.2f}")

        # --- Record a random episode ---
        random_ep = random.randint(0, num_tests - 1)
        print(f"🎥 Recording episode {random_ep + 1} for {model_file}...")

        # Recreate env with video recording enabled
        video_env = gym.make(env_name, render_mode="rgb_array")
        if isinstance(video_env.action_space, gym.spaces.Box):
            video_env = DiscretizeActionWrapper(video_env, num_discrete_actions=num_discrete_actions)

        video_env = RecordVideo(
            video_env,
            video_folder=f"videos/{env_name}/{model_file[:-4]}",
            episode_trigger=lambda e: True,
            name_prefix=f"{model_file[:-4]}"
        )

        # Replay the chosen episode
        video_env.reset()
        done = False
        state, _ = video_env.reset()

        while not done:
            state_tensor = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)
            with torch.no_grad():
                q_values = model(state_tensor)
                action = torch.argmax(q_values).item()
            next_state, _, terminated, truncated, _ = video_env.step(action)
            done = terminated or truncated
            state = next_state

        video_env.close()
        print(f"🎞️ Saved video for {model_file} at ./videos/{env_name}/{model_file[:-4]}/")

    print("\n🏁 All models tested and recorded successfully.")

In [10]:
record_models_in_folder(
    models_folder="./Acrobot models",
    env_name="Acrobot-v1",
    num_discrete_actions=5,
    num_tests=100
)


🎬 Testing and recording model: Acrobot_DQN_batch-128.pth
  Episode 1/100: reward = -93.00
  Episode 2/100: reward = -143.00
  Episode 3/100: reward = -91.00
  Episode 4/100: reward = -98.00
  Episode 5/100: reward = -114.00
  Episode 6/100: reward = -110.00
  Episode 7/100: reward = -125.00
  Episode 8/100: reward = -82.00
  Episode 9/100: reward = -100.00
  Episode 10/100: reward = -85.00
  Episode 11/100: reward = -100.00
  Episode 12/100: reward = -89.00
  Episode 13/100: reward = -86.00
  Episode 14/100: reward = -96.00
  Episode 15/100: reward = -98.00
  Episode 16/100: reward = -175.00
  Episode 17/100: reward = -119.00
  Episode 18/100: reward = -82.00
  Episode 19/100: reward = -129.00
  Episode 20/100: reward = -108.00
  Episode 21/100: reward = -92.00
  Episode 22/100: reward = -96.00
  Episode 23/100: reward = -87.00
  Episode 24/100: reward = -81.00
  Episode 25/100: reward = -98.00
  Episode 26/100: reward = -113.00
  Episode 27/100: reward = -97.00
  Episode 28/100: rewa

d:\materials for college\Fall 2025-2026\Reinforcement Learning\assignments\assignment 2\.venv\Lib\site-packages\pygame\pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


🎞️ Saved video for Acrobot_DQN_batch-128.pth at ./videos/Acrobot-v1/Acrobot_DQN_batch-128/

🎬 Testing and recording model: Acrobot_DQN_batch-256.pth
  Episode 1/100: reward = -69.00
  Episode 2/100: reward = -75.00
  Episode 3/100: reward = -73.00
  Episode 4/100: reward = -128.00
  Episode 5/100: reward = -152.00
  Episode 6/100: reward = -92.00
  Episode 7/100: reward = -114.00
  Episode 8/100: reward = -83.00
  Episode 9/100: reward = -90.00
  Episode 10/100: reward = -104.00
  Episode 11/100: reward = -61.00
  Episode 12/100: reward = -97.00
  Episode 13/100: reward = -76.00
  Episode 14/100: reward = -90.00
  Episode 15/100: reward = -87.00
  Episode 16/100: reward = -136.00
  Episode 17/100: reward = -89.00
  Episode 18/100: reward = -84.00
  Episode 19/100: reward = -114.00
  Episode 20/100: reward = -74.00
  Episode 21/100: reward = -77.00
  Episode 22/100: reward = -94.00
  Episode 23/100: reward = -87.00
  Episode 24/100: reward = -78.00
  Episode 25/100: reward = -109.00
  E

d:\materials for college\Fall 2025-2026\Reinforcement Learning\assignments\assignment 2\.venv\Lib\site-packages\gymnasium\wrappers\rendering.py:293: UserWarning: WARN: Overwriting existing videos at d:\materials for college\Fall 2025-2026\Reinforcement Learning\assignments\assignment 2\videos\Acrobot-v1\Acrobot_DQN_gamma-0.9 folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


🎞️ Saved video for Acrobot_DQN_gamma-0.9.pth at ./videos/Acrobot-v1/Acrobot_DQN_gamma-0.9/

🎬 Testing and recording model: Acrobot_DQN_gamma-0.99.pth
  Episode 1/100: reward = -82.00
  Episode 2/100: reward = -195.00
  Episode 3/100: reward = -103.00
  Episode 4/100: reward = -63.00
  Episode 5/100: reward = -87.00
  Episode 6/100: reward = -124.00
  Episode 7/100: reward = -94.00
  Episode 8/100: reward = -70.00
  Episode 9/100: reward = -86.00
  Episode 10/100: reward = -69.00
  Episode 11/100: reward = -73.00
  Episode 12/100: reward = -103.00
  Episode 13/100: reward = -76.00
  Episode 14/100: reward = -93.00
  Episode 15/100: reward = -86.00
  Episode 16/100: reward = -74.00
  Episode 17/100: reward = -88.00
  Episode 18/100: reward = -72.00
  Episode 19/100: reward = -78.00
  Episode 20/100: reward = -78.00
  Episode 21/100: reward = -78.00
  Episode 22/100: reward = -83.00
  Episode 23/100: reward = -167.00
  Episode 24/100: reward = -88.00
  Episode 25/100: reward = -62.00
  Ep

d:\materials for college\Fall 2025-2026\Reinforcement Learning\assignments\assignment 2\.venv\Lib\site-packages\gymnasium\wrappers\rendering.py:293: UserWarning: WARN: Overwriting existing videos at d:\materials for college\Fall 2025-2026\Reinforcement Learning\assignments\assignment 2\videos\Acrobot-v1\Acrobot_DQN_gamma-0.99 folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


🎞️ Saved video for Acrobot_DQN_gamma-0.99.pth at ./videos/Acrobot-v1/Acrobot_DQN_gamma-0.99/

🎬 Testing and recording model: Acrobot_DQN_lr-0.0003.pth
  Episode 1/100: reward = -76.00
  Episode 2/100: reward = -110.00
  Episode 3/100: reward = -81.00
  Episode 4/100: reward = -88.00
  Episode 5/100: reward = -87.00
  Episode 6/100: reward = -76.00
  Episode 7/100: reward = -86.00
  Episode 8/100: reward = -77.00
  Episode 9/100: reward = -73.00
  Episode 10/100: reward = -77.00
  Episode 11/100: reward = -101.00
  Episode 12/100: reward = -81.00
  Episode 13/100: reward = -93.00
  Episode 14/100: reward = -72.00
  Episode 15/100: reward = -86.00
  Episode 16/100: reward = -71.00
  Episode 17/100: reward = -75.00
  Episode 18/100: reward = -69.00
  Episode 19/100: reward = -88.00
  Episode 20/100: reward = -73.00
  Episode 21/100: reward = -73.00
  Episode 22/100: reward = -85.00
  Episode 23/100: reward = -94.00
  Episode 24/100: reward = -81.00
  Episode 25/100: reward = -77.00
  Epis

d:\materials for college\Fall 2025-2026\Reinforcement Learning\assignments\assignment 2\.venv\Lib\site-packages\gymnasium\wrappers\rendering.py:293: UserWarning: WARN: Overwriting existing videos at d:\materials for college\Fall 2025-2026\Reinforcement Learning\assignments\assignment 2\videos\Acrobot-v1\Acrobot_DQN_lr-0.001 folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


🎞️ Saved video for Acrobot_DQN_lr-0.001.pth at ./videos/Acrobot-v1/Acrobot_DQN_lr-0.001/

🎬 Testing and recording model: Acrobot_DQN_lr-0.01.pth
  Episode 1/100: reward = -101.00
  Episode 2/100: reward = -73.00
  Episode 3/100: reward = -110.00
  Episode 4/100: reward = -70.00
  Episode 5/100: reward = -80.00
  Episode 6/100: reward = -92.00
  Episode 7/100: reward = -83.00
  Episode 8/100: reward = -70.00
  Episode 9/100: reward = -71.00
  Episode 10/100: reward = -96.00
  Episode 11/100: reward = -81.00
  Episode 12/100: reward = -76.00
  Episode 13/100: reward = -82.00
  Episode 14/100: reward = -90.00
  Episode 15/100: reward = -69.00
  Episode 16/100: reward = -72.00
  Episode 17/100: reward = -80.00
  Episode 18/100: reward = -92.00
  Episode 19/100: reward = -77.00
  Episode 20/100: reward = -81.00
  Episode 21/100: reward = -62.00
  Episode 22/100: reward = -71.00
  Episode 23/100: reward = -80.00
  Episode 24/100: reward = -70.00
  Episode 25/100: reward = -62.00
  Episode 26

In [19]:
record_models_in_folder(
    models_folder="./cartPole models",
    env_name="CartPole-v1",
    num_discrete_actions=5,
    num_tests=100
)


🎬 Testing and recording model: CartPole_DQN_batch-128.pth
  Episode 1/100: reward = 500.00
  Episode 2/100: reward = 500.00
  Episode 3/100: reward = 500.00
  Episode 4/100: reward = 500.00
  Episode 5/100: reward = 500.00
  Episode 6/100: reward = 500.00
  Episode 7/100: reward = 500.00
  Episode 8/100: reward = 500.00
  Episode 9/100: reward = 500.00
  Episode 10/100: reward = 500.00
  Episode 11/100: reward = 500.00
  Episode 12/100: reward = 500.00
  Episode 13/100: reward = 500.00
  Episode 14/100: reward = 500.00
  Episode 15/100: reward = 500.00
  Episode 16/100: reward = 500.00
  Episode 17/100: reward = 500.00
  Episode 18/100: reward = 500.00
  Episode 19/100: reward = 500.00
  Episode 20/100: reward = 500.00
  Episode 21/100: reward = 500.00
  Episode 22/100: reward = 500.00
  Episode 23/100: reward = 500.00
  Episode 24/100: reward = 500.00
  Episode 25/100: reward = 500.00
  Episode 26/100: reward = 500.00
  Episode 27/100: reward = 500.00
  Episode 28/100: reward = 500.0

d:\materials for college\Fall 2025-2026\Reinforcement Learning\assignments\assignment 2\.venv\Lib\site-packages\gymnasium\wrappers\rendering.py:293: UserWarning: WARN: Overwriting existing videos at d:\materials for college\Fall 2025-2026\Reinforcement Learning\assignments\assignment 2\videos\CartPole-v1\CartPole_DQN_eps-1000 folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


🎞️ Saved video for CartPole_DQN_eps-1000.pth at ./videos/CartPole-v1/CartPole_DQN_eps-1000/

🎬 Testing and recording model: CartPole_DQN_eps-5000.pth
  Episode 1/100: reward = 500.00
  Episode 2/100: reward = 500.00
  Episode 3/100: reward = 500.00
  Episode 4/100: reward = 500.00
  Episode 5/100: reward = 500.00
  Episode 6/100: reward = 500.00
  Episode 7/100: reward = 500.00
  Episode 8/100: reward = 500.00
  Episode 9/100: reward = 500.00
  Episode 10/100: reward = 500.00
  Episode 11/100: reward = 500.00
  Episode 12/100: reward = 500.00
  Episode 13/100: reward = 500.00
  Episode 14/100: reward = 500.00
  Episode 15/100: reward = 500.00
  Episode 16/100: reward = 500.00
  Episode 17/100: reward = 500.00
  Episode 18/100: reward = 500.00
  Episode 19/100: reward = 500.00
  Episode 20/100: reward = 500.00
  Episode 21/100: reward = 500.00
  Episode 22/100: reward = 500.00
  Episode 23/100: reward = 500.00
  Episode 24/100: reward = 500.00
  Episode 25/100: reward = 500.00
  Episode

d:\materials for college\Fall 2025-2026\Reinforcement Learning\assignments\assignment 2\.venv\Lib\site-packages\gymnasium\wrappers\rendering.py:293: UserWarning: WARN: Overwriting existing videos at d:\materials for college\Fall 2025-2026\Reinforcement Learning\assignments\assignment 2\videos\CartPole-v1\CartPole_DQN_eps-5000 folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


🎞️ Saved video for CartPole_DQN_eps-5000.pth at ./videos/CartPole-v1/CartPole_DQN_eps-5000/

🎬 Testing and recording model: CartPole_DQN_gamma-0.1.pth
  Episode 1/100: reward = 9.00
  Episode 2/100: reward = 10.00
  Episode 3/100: reward = 9.00
  Episode 4/100: reward = 9.00
  Episode 5/100: reward = 10.00
  Episode 6/100: reward = 9.00
  Episode 7/100: reward = 9.00
  Episode 8/100: reward = 9.00
  Episode 9/100: reward = 11.00
  Episode 10/100: reward = 10.00
  Episode 11/100: reward = 9.00
  Episode 12/100: reward = 9.00
  Episode 13/100: reward = 11.00
  Episode 14/100: reward = 9.00
  Episode 15/100: reward = 10.00
  Episode 16/100: reward = 9.00
  Episode 17/100: reward = 10.00
  Episode 18/100: reward = 8.00
  Episode 19/100: reward = 10.00
  Episode 20/100: reward = 10.00
  Episode 21/100: reward = 10.00
  Episode 22/100: reward = 11.00
  Episode 23/100: reward = 9.00
  Episode 24/100: reward = 9.00
  Episode 25/100: reward = 9.00
  Episode 26/100: reward = 10.00
  Episode 27/1

...

🎞️ Saved video for CartPole_DQN_lr-0.001.pth at ./videos/CartPole-v1/CartPole_DQN_lr-0.001/

🎬 Testing and recording model: CartPole_DQN_lr-0.01.pth
  Episode 1/100: reward = 500.00
  Episode 2/100: reward = 500.00
  Episode 3/100: reward = 500.00
  Episode 4/100: reward = 500.00
  Episode 5/100: reward = 500.00
  Episode 6/100: reward = 500.00
  Episode 7/100: reward = 500.00
  Episode 8/100: reward = 500.00
  Episode 9/100: reward = 500.00
  Episode 10/100: reward = 500.00
  Episode 11/100: reward = 500.00
  Episode 12/100: reward = 500.00
  Episode 13/100: reward = 500.00
  Episode 14/100: reward = 500.00
  Episode 15/100: reward = 500.00
  Episode 16/100: reward = 500.00
  Episode 17/100: reward = 500.00
  Episode 18/100: reward = 500.00
  Episode 19/100: reward = 500.00
  Episode 20/100: reward = 500.00
  Episode 21/100: reward = 500.00
  Episode 22/100: reward = 500.00
  Episode 23/100: reward = 500.00
  Episode 24/100: reward = 500.00
  Episode 25/100: reward = 500.00
  Episode 

d:\materials for college\Fall 2025-2026\Reinforcement Learning\assignments\assignment 2\.venv\Lib\site-packages\gymnasium\wrappers\rendering.py:293: UserWarning: WARN: Overwriting existing videos at d:\materials for college\Fall 2025-2026\Reinforcement Learning\assignments\assignment 2\videos\CartPole-v1\CartPole_DQN_mem-10000 folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


🎞️ Saved video for CartPole_DQN_mem-10000.pth at ./videos/CartPole-v1/CartPole_DQN_mem-10000/

🎬 Testing and recording model: CartPole_DQN_mem-50000.pth
  Episode 1/100: reward = 189.00
  Episode 2/100: reward = 171.00
  Episode 3/100: reward = 159.00
  Episode 4/100: reward = 168.00
  Episode 5/100: reward = 156.00
  Episode 6/100: reward = 171.00
  Episode 7/100: reward = 177.00
  Episode 8/100: reward = 174.00
  Episode 9/100: reward = 155.00
  Episode 10/100: reward = 160.00
  Episode 11/100: reward = 160.00
  Episode 12/100: reward = 175.00
  Episode 13/100: reward = 170.00
  Episode 14/100: reward = 165.00
  Episode 15/100: reward = 154.00
  Episode 16/100: reward = 159.00
  Episode 17/100: reward = 169.00
  Episode 18/100: reward = 174.00
  Episode 19/100: reward = 158.00
  Episode 20/100: reward = 161.00
  Episode 21/100: reward = 162.00
  Episode 22/100: reward = 169.00
  Episode 23/100: reward = 180.00
  Episode 24/100: reward = 174.00
  Episode 25/100: reward = 164.00
  Epis

d:\materials for college\Fall 2025-2026\Reinforcement Learning\assignments\assignment 2\.venv\Lib\site-packages\gymnasium\wrappers\rendering.py:293: UserWarning: WARN: Overwriting existing videos at d:\materials for college\Fall 2025-2026\Reinforcement Learning\assignments\assignment 2\videos\CartPole-v1\CartPole_DQN_mem-50000 folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


🎞️ Saved video for CartPole_DQN_mem-50000.pth at ./videos/CartPole-v1/CartPole_DQN_mem-50000/

🏁 All models tested and recorded successfully.


In [11]:
record_models_in_folder(
    models_folder="./MountainCar models",
    env_name="MountainCar-v0",
    num_discrete_actions=5,
    num_tests=100
)


🎬 Testing and recording model: MountainCar_DQN_batch-128.pth
  Episode 1/100: reward = -200.00
  Episode 2/100: reward = -200.00
  Episode 3/100: reward = -200.00
  Episode 4/100: reward = -200.00
  Episode 5/100: reward = -200.00
  Episode 6/100: reward = -200.00
  Episode 7/100: reward = -200.00
  Episode 8/100: reward = -200.00
  Episode 9/100: reward = -200.00
  Episode 10/100: reward = -200.00
  Episode 11/100: reward = -200.00
  Episode 12/100: reward = -200.00
  Episode 13/100: reward = -200.00
  Episode 14/100: reward = -200.00
  Episode 15/100: reward = -200.00
  Episode 16/100: reward = -200.00
  Episode 17/100: reward = -200.00
  Episode 18/100: reward = -200.00
  Episode 19/100: reward = -200.00
  Episode 20/100: reward = -200.00
  Episode 21/100: reward = -200.00
  Episode 22/100: reward = -200.00
  Episode 23/100: reward = -200.00
  Episode 24/100: reward = -200.00
  Episode 25/100: reward = -200.00
  Episode 26/100: reward = -200.00
  Episode 27/100: reward = -200.00
  

d:\materials for college\Fall 2025-2026\Reinforcement Learning\assignments\assignment 2\.venv\Lib\site-packages\gymnasium\wrappers\rendering.py:293: UserWarning: WARN: Overwriting existing videos at d:\materials for college\Fall 2025-2026\Reinforcement Learning\assignments\assignment 2\videos\MountainCar-v0\MountainCar_DQN_eps-1000 folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


🎞️ Saved video for MountainCar_DQN_eps-1000.pth at ./videos/MountainCar-v0/MountainCar_DQN_eps-1000/

🎬 Testing and recording model: MountainCar_DQN_eps-2500.pth
  Episode 1/100: reward = -143.00
  Episode 2/100: reward = -175.00
  Episode 3/100: reward = -144.00
  Episode 4/100: reward = -111.00
  Episode 5/100: reward = -91.00
  Episode 6/100: reward = -96.00
  Episode 7/100: reward = -92.00
  Episode 8/100: reward = -93.00
  Episode 9/100: reward = -87.00
  Episode 10/100: reward = -97.00
  Episode 11/100: reward = -83.00
  Episode 12/100: reward = -98.00
  Episode 13/100: reward = -143.00
  Episode 14/100: reward = -171.00
  Episode 15/100: reward = -143.00
  Episode 16/100: reward = -144.00
  Episode 17/100: reward = -83.00
  Episode 18/100: reward = -143.00
  Episode 19/100: reward = -113.00
  Episode 20/100: reward = -84.00
  Episode 21/100: reward = -167.00
  Episode 22/100: reward = -143.00
  Episode 23/100: reward = -143.00
  Episode 24/100: reward = -87.00
  Episode 25/100: 

d:\materials for college\Fall 2025-2026\Reinforcement Learning\assignments\assignment 2\.venv\Lib\site-packages\gymnasium\wrappers\rendering.py:293: UserWarning: WARN: Overwriting existing videos at d:\materials for college\Fall 2025-2026\Reinforcement Learning\assignments\assignment 2\videos\MountainCar-v0\MountainCar_DQN_eps-5000 folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


🎞️ Saved video for MountainCar_DQN_eps-5000.pth at ./videos/MountainCar-v0/MountainCar_DQN_eps-5000/

🎬 Testing and recording model: MountainCar_DQN_gamma-0.1.pth
  Episode 1/100: reward = -200.00
  Episode 2/100: reward = -200.00
  Episode 3/100: reward = -200.00
  Episode 4/100: reward = -200.00
  Episode 5/100: reward = -200.00
  Episode 6/100: reward = -200.00
  Episode 7/100: reward = -200.00
  Episode 8/100: reward = -200.00
  Episode 9/100: reward = -200.00
  Episode 10/100: reward = -200.00
  Episode 11/100: reward = -200.00
  Episode 12/100: reward = -200.00
  Episode 13/100: reward = -200.00
  Episode 14/100: reward = -200.00
  Episode 15/100: reward = -200.00
  Episode 16/100: reward = -200.00
  Episode 17/100: reward = -200.00
  Episode 18/100: reward = -200.00
  Episode 19/100: reward = -200.00
  Episode 20/100: reward = -200.00
  Episode 21/100: reward = -200.00
  Episode 22/100: reward = -200.00
  Episode 23/100: reward = -200.00
  Episode 24/100: reward = -200.00
  Epis

...

🎞️ Saved video for MountainCar_DQN_lr-0.001.pth at ./videos/MountainCar-v0/MountainCar_DQN_lr-0.001/

🎬 Testing and recording model: MountainCar_DQN_lr-0.01.pth
  Episode 1/100: reward = -200.00
  Episode 2/100: reward = -200.00
  Episode 3/100: reward = -200.00
  Episode 4/100: reward = -200.00
  Episode 5/100: reward = -200.00
  Episode 6/100: reward = -200.00
  Episode 7/100: reward = -200.00
  Episode 8/100: reward = -200.00
  Episode 9/100: reward = -200.00
  Episode 10/100: reward = -200.00
  Episode 11/100: reward = -200.00
  Episode 12/100: reward = -200.00
  Episode 13/100: reward = -200.00
  Episode 14/100: reward = -200.00
  Episode 15/100: reward = -200.00
  Episode 16/100: reward = -200.00
  Episode 17/100: reward = -200.00
  Episode 18/100: reward = -200.00
  Episode 19/100: reward = -200.00
  Episode 20/100: reward = -200.00
  Episode 21/100: reward = -200.00
  Episode 22/100: reward = -200.00
  Episode 23/100: reward = -200.00
  Episode 24/100: reward = -200.00
  Episod

d:\materials for college\Fall 2025-2026\Reinforcement Learning\assignments\assignment 2\.venv\Lib\site-packages\gymnasium\wrappers\rendering.py:293: UserWarning: WARN: Overwriting existing videos at d:\materials for college\Fall 2025-2026\Reinforcement Learning\assignments\assignment 2\videos\MountainCar-v0\MountainCar_DQN_mem-10000 folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


🎞️ Saved video for MountainCar_DQN_mem-10000 .pth at ./videos/MountainCar-v0/MountainCar_DQN_mem-10000 /

🎬 Testing and recording model: MountainCar_DQN_mem-500.pth
  Episode 1/100: reward = -200.00
  Episode 2/100: reward = -200.00
  Episode 3/100: reward = -174.00
  Episode 4/100: reward = -182.00
  Episode 5/100: reward = -166.00
  Episode 6/100: reward = -92.00
  Episode 7/100: reward = -90.00
  Episode 8/100: reward = -180.00
  Episode 9/100: reward = -89.00
  Episode 10/100: reward = -90.00
  Episode 11/100: reward = -135.00
  Episode 12/100: reward = -91.00
  Episode 13/100: reward = -185.00
  Episode 14/100: reward = -110.00
  Episode 15/100: reward = -166.00
  Episode 16/100: reward = -200.00
  Episode 17/100: reward = -197.00
  Episode 18/100: reward = -170.00
  Episode 19/100: reward = -200.00
  Episode 20/100: reward = -103.00
  Episode 21/100: reward = -167.00
  Episode 22/100: reward = -136.00
  Episode 23/100: reward = -90.00
  Episode 24/100: reward = -103.00
  Episode 

d:\materials for college\Fall 2025-2026\Reinforcement Learning\assignments\assignment 2\.venv\Lib\site-packages\gymnasium\wrappers\rendering.py:293: UserWarning: WARN: Overwriting existing videos at d:\materials for college\Fall 2025-2026\Reinforcement Learning\assignments\assignment 2\videos\MountainCar-v0\MountainCar_DQN_mem-50000 folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


🎞️ Saved video for MountainCar_DQN_mem-50000.pth at ./videos/MountainCar-v0/MountainCar_DQN_mem-50000/

🏁 All models tested and recorded successfully.


In [20]:
record_models_in_folder(
    models_folder="./pendulum Models",
    env_name="Pendulum-v1",
    num_discrete_actions=5,
    num_tests=100
)


🎬 Testing and recording model: Pendulum_DQN_eps-1000.pth
  Episode 1/100: reward = -1.05
  Episode 2/100: reward = -122.19
  Episode 3/100: reward = -116.88
  Episode 4/100: reward = -118.88
  Episode 5/100: reward = -225.83
  Episode 6/100: reward = -1.04
  Episode 7/100: reward = -236.11
  Episode 8/100: reward = -119.33
  Episode 9/100: reward = -233.71
  Episode 10/100: reward = -238.68
  Episode 11/100: reward = -228.69
  Episode 12/100: reward = -116.75
  Episode 13/100: reward = -125.85
  Episode 14/100: reward = -120.37
  Episode 15/100: reward = -119.82
  Episode 16/100: reward = -233.81
  Episode 17/100: reward = -128.60
  Episode 18/100: reward = -128.07
  Episode 19/100: reward = -125.38
  Episode 20/100: reward = -2.37
  Episode 21/100: reward = -115.35
  Episode 22/100: reward = -1.21
  Episode 23/100: reward = -114.12
  Episode 24/100: reward = -120.35
  Episode 25/100: reward = -116.36
  Episode 26/100: reward = -2.27
  Episode 27/100: reward = -311.96
  Episode 28/100

d:\materials for college\Fall 2025-2026\Reinforcement Learning\assignments\assignment 2\.venv\Lib\site-packages\gymnasium\wrappers\rendering.py:293: UserWarning: WARN: Overwriting existing videos at d:\materials for college\Fall 2025-2026\Reinforcement Learning\assignments\assignment 2\videos\Pendulum-v1\Pendulum_DQN_eps-1000 folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


🎞️ Saved video for Pendulum_DQN_eps-1000.pth at ./videos/Pendulum-v1/Pendulum_DQN_eps-1000/

🎬 Testing and recording model: Pendulum_DQN_eps-5000.pth
  Episode 1/100: reward = -123.74
  Episode 2/100: reward = -232.52
  Episode 3/100: reward = -118.73
  Episode 4/100: reward = -230.59
  Episode 5/100: reward = -306.82
  Episode 6/100: reward = -124.39
  Episode 7/100: reward = -231.81
  Episode 8/100: reward = -124.10
  Episode 9/100: reward = -126.65
  Episode 10/100: reward = -119.60
  Episode 11/100: reward = -1.35
  Episode 12/100: reward = -231.99
  Episode 13/100: reward = -232.13
  Episode 14/100: reward = -122.06
  Episode 15/100: reward = -120.44
  Episode 16/100: reward = -0.91
  Episode 17/100: reward = -126.68
  Episode 18/100: reward = -123.03
  Episode 19/100: reward = -115.99
  Episode 20/100: reward = -121.69
  Episode 21/100: reward = -123.99
  Episode 22/100: reward = -228.88
  Episode 23/100: reward = -232.73
  Episode 24/100: reward = -114.20
  Episode 25/100: rewar

d:\materials for college\Fall 2025-2026\Reinforcement Learning\assignments\assignment 2\.venv\Lib\site-packages\gymnasium\wrappers\rendering.py:293: UserWarning: WARN: Overwriting existing videos at d:\materials for college\Fall 2025-2026\Reinforcement Learning\assignments\assignment 2\videos\Pendulum-v1\Pendulum_DQN_eps-5000 folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


🎞️ Saved video for Pendulum_DQN_eps-5000.pth at ./videos/Pendulum-v1/Pendulum_DQN_eps-5000/

🎬 Testing and recording model: Pendulum_DQN_gamma-0.1.pth
  Episode 1/100: reward = -1512.11
  Episode 2/100: reward = -1511.23
  Episode 3/100: reward = -1518.56
  Episode 4/100: reward = -1357.19
  Episode 5/100: reward = -1485.71
  Episode 6/100: reward = -1494.52
  Episode 7/100: reward = -1437.19
  Episode 8/100: reward = -1467.10
  Episode 9/100: reward = -1456.63
  Episode 10/100: reward = -1385.27
  Episode 11/100: reward = -1514.94
  Episode 12/100: reward = -1509.34
  Episode 13/100: reward = -1455.04
  Episode 14/100: reward = -1369.22
  Episode 15/100: reward = -1494.89
  Episode 16/100: reward = -1476.26
  Episode 17/100: reward = -1469.93
  Episode 18/100: reward = -1505.67
  Episode 19/100: reward = -1459.79
  Episode 20/100: reward = -1505.03
  Episode 21/100: reward = -1468.80
  Episode 22/100: reward = -1519.26
  Episode 23/100: reward = -1504.73
  Episode 24/100: reward = -14

...

🎞️ Saved video for Pendulum_DQN_lr-0.001.pth at ./videos/Pendulum-v1/Pendulum_DQN_lr-0.001/

🎬 Testing and recording model: Pendulum_DQN_lr-0.01.pth
  Episode 1/100: reward = -223.55
  Episode 2/100: reward = -243.18
  Episode 3/100: reward = -231.45
  Episode 4/100: reward = -230.66
  Episode 5/100: reward = -125.01
  Episode 6/100: reward = -353.79
  Episode 7/100: reward = -252.57
  Episode 8/100: reward = -249.73
  Episode 9/100: reward = -126.42
  Episode 10/100: reward = -125.52
  Episode 11/100: reward = -123.65
  Episode 12/100: reward = -244.20
  Episode 13/100: reward = -223.84
  Episode 14/100: reward = -222.95
  Episode 15/100: reward = -2.88
  Episode 16/100: reward = -0.98
  Episode 17/100: reward = -121.94
  Episode 18/100: reward = -125.03
  Episode 19/100: reward = -123.67
  Episode 20/100: reward = -357.78
  Episode 21/100: reward = -233.65
  Episode 22/100: reward = -235.43
  Episode 23/100: reward = -261.47
  Episode 24/100: reward = -249.80
  Episode 25/100: reward

d:\materials for college\Fall 2025-2026\Reinforcement Learning\assignments\assignment 2\.venv\Lib\site-packages\gymnasium\wrappers\rendering.py:293: UserWarning: WARN: Overwriting existing videos at d:\materials for college\Fall 2025-2026\Reinforcement Learning\assignments\assignment 2\videos\Pendulum-v1\Pendulum_DQN_mem-10000 folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


🎞️ Saved video for Pendulum_DQN_mem-10000.pth at ./videos/Pendulum-v1/Pendulum_DQN_mem-10000/

🎬 Testing and recording model: Pendulum_DQN_mem-50000.pth
  Episode 1/100: reward = -233.44
  Episode 2/100: reward = -224.62
  Episode 3/100: reward = -237.96
  Episode 4/100: reward = -0.90
  Episode 5/100: reward = -119.13
  Episode 6/100: reward = -121.36
  Episode 7/100: reward = -226.21
  Episode 8/100: reward = -115.68
  Episode 9/100: reward = -116.81
  Episode 10/100: reward = -117.48
  Episode 11/100: reward = -120.59
  Episode 12/100: reward = -123.47
  Episode 13/100: reward = -121.86
  Episode 14/100: reward = -233.05
  Episode 15/100: reward = -124.34
  Episode 16/100: reward = -123.24
  Episode 17/100: reward = -222.96
  Episode 18/100: reward = -117.64
  Episode 19/100: reward = -327.20
  Episode 20/100: reward = -123.64
  Episode 21/100: reward = -239.40
  Episode 22/100: reward = -117.51
  Episode 23/100: reward = -119.16
  Episode 24/100: reward = -229.22
  Episode 25/100: 

d:\materials for college\Fall 2025-2026\Reinforcement Learning\assignments\assignment 2\.venv\Lib\site-packages\gymnasium\wrappers\rendering.py:293: UserWarning: WARN: Overwriting existing videos at d:\materials for college\Fall 2025-2026\Reinforcement Learning\assignments\assignment 2\videos\Pendulum-v1\Pendulum_DQN_mem-50000 folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


🎞️ Saved video for Pendulum_DQN_mem-50000.pth at ./videos/Pendulum-v1/Pendulum_DQN_mem-50000/

🏁 All models tested and recorded successfully.


In [15]:
def test_models_folder_with_wandb(
    models_folder,
    env_name,
    num_tests=100,
    num_discrete_actions=5,
    wandb_project="RL A2 tests",
    seed=None
):
    if seed is not None:
        random.seed(seed)
        np.random.seed(seed)
        torch.manual_seed(seed)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model_files = sorted([f for f in os.listdir(models_folder) if f.endswith(".pth")])
    if not model_files:
        print("No .pth files found in", models_folder)
        return

    for model_file in model_files:
        model_path = os.path.join(models_folder, model_file)
        run_name = f"test_{os.path.splitext(model_file)[0]}_{int(time.time())}"
        print(f"\n=== Model: {model_file} ===")
        print("W&B run:", run_name)

        # Init W&B run
        run = wandb.init(project=wandb_project, name=run_name, reinit=True, config={
            "env_name": env_name,
            "model_file": model_file,
            "num_tests": num_tests,
            "num_discrete_actions": num_discrete_actions
        })

        # Build environment
        base_env = gym.make(env_name, render_mode="rgb_array")
        if isinstance(base_env.action_space, gym.spaces.Box):
            env = DiscretizeActionWrapper(base_env, num_discrete_actions=num_discrete_actions)
        else:
            env = base_env

        # Load model (infer sizes from env reset)
        sample_state, _ = env.reset()
        env.render()
        n_observations = len(sample_state)
        n_actions = env.action_space.n

        model = DQN(n_observations, n_actions).to(device)
        model.load_state_dict(torch.load(model_path, map_location=device))
        model.eval()

        # --- Run multiple test episodes ---
        episode_rewards = []
        for ep in range(num_tests):
            state, _ = env.reset()
            done = False
            total_reward = 0
            steps = 0

            while not done:
                state_tensor = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)
                with torch.no_grad():
                    q_values = model(state_tensor)
                    action = torch.argmax(q_values).item()

                next_state, reward, terminated, truncated, _ = env.step(action)
                total_reward += reward
                steps += 1
                done = terminated or truncated
                state = next_state

            episode_rewards.append(total_reward)
            wandb.log({"episode": ep, "episode_reward": total_reward, "steps": steps})
            print(f"  Ep {ep+1}/{num_tests}: reward={total_reward:.2f} steps={steps}")

        avg_reward = float(np.mean(episode_rewards))
        best_reward = float(np.max(episode_rewards))
        worst_reward = float(np.min(episode_rewards))
        wandb.log({"avg_reward": avg_reward, "best_reward": best_reward, "worst_reward": worst_reward})
        print(f"Finished evaluations. avg={avg_reward:.2f} best={best_reward:.2f} worst={worst_reward:.2f}")

        # --- Record a single random episode ---
        record_idx = random.randrange(0, num_tests)
        print(f"Recording random episode index {record_idx} for model {model_file}")

        # Wrap fresh env for recording
        video_folder = f"videos/{env_name}/{os.path.splitext(model_file)[0]}"
        os.makedirs(video_folder, exist_ok=True)

        video_base = gym.make(env_name, render_mode="rgb_array")
        if isinstance(video_base.action_space, gym.spaces.Box):
            video_env = DiscretizeActionWrapper(video_base, num_discrete_actions=num_discrete_actions)
        else:
            video_env = video_base

        # Record only **the next episode**
        video_env = RecordVideo(
            video_env,
            video_folder=video_folder,
            episode_trigger=lambda e: True,  # record the first episode run
            name_prefix=os.path.splitext(model_file)[0],
            disable_logger=True
        )

        state, _ = video_env.reset()
        done = False
        total_reward = 0
        steps = 0
        testing_start_time = time.time()
        while not done:
            state_tensor = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)
            with torch.no_grad():
                q_values = model(state_tensor)
                action = torch.argmax(q_values).item()

            next_state, reward, terminated, truncated, _ = video_env.step(action)
            total_reward += reward
            steps += 1
            done = terminated or truncated
            state = next_state

        video_env.close()

        # Log video to W&B
        vid_files = [f for f in os.listdir(video_folder) if f.endswith(".mp4")]
        if vid_files:
            vid_files = sorted(vid_files, key=lambda p: os.path.getmtime(os.path.join(video_folder, p)), reverse=True)
            video_path = os.path.join(video_folder, vid_files[0])
            print("Recorded video:", video_path)
            wandb.log({"test_video": wandb.Video(video_path, fps=30, format="mp4")})
        else:
            print("Warning: no video file found in", video_folder)

        testing_end_time = time.time()
        testing_duration = testing_end_time - testing_start_time
        print(f"Recording complete. Total reward: {total_reward:.2f}, Steps:{steps}, Duration: {testing_duration:.2f} seconds")
        # Log recorded episode stats
        wandb.log({
            "recorded_episode_index": record_idx,
            "recorded_episode_reward": total_reward,
            "recorded_episode_steps": steps,
            "recorded_episode_duration_sec": testing_duration
        })

        run.finish()
        time.sleep(0.5)

In [21]:
test_models_folder_with_wandb(
    env_name="CartPole-v1",
    num_discrete_actions=5,
    models_folder="./cartPole models",
    num_tests=100,
    wandb_project="RL A2 tests"
)


=== Model: CartPole_DQN_batch-128.pth ===
W&B run: test_CartPole_DQN_batch-128_1763011249


wandb: ERROR The nbformat package was not found. It is required to save notebook history.


avg_reward,▁
best_reward,▁
episode,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇██
episode_reward,██▇▇██▇█▇▇▇██▇▇▁█▁█▁██▁█▇▇█▁██▁██▁██▇▁██
steps,▁▁▁▁▂▁▁▁█▁▂█▁▁▁█▂▁███▂▁▁▁▂▁▁▂▁▁▁▂▂▁▁██▂▂
worst_reward,▁
avg_reward,-156.36
best_reward,-140
episode,99
episode_reward,-143
steps,143


  Ep 1/100: reward=500.00 steps=500
  Ep 2/100: reward=500.00 steps=500
  Ep 3/100: reward=500.00 steps=500
  Ep 4/100: reward=500.00 steps=500
  Ep 5/100: reward=500.00 steps=500
  Ep 6/100: reward=500.00 steps=500
  Ep 7/100: reward=500.00 steps=500
  Ep 8/100: reward=500.00 steps=500
  Ep 9/100: reward=500.00 steps=500
  Ep 10/100: reward=500.00 steps=500
  Ep 11/100: reward=500.00 steps=500
  Ep 12/100: reward=500.00 steps=500
  Ep 13/100: reward=500.00 steps=500
  Ep 14/100: reward=500.00 steps=500
  Ep 15/100: reward=500.00 steps=500
  Ep 16/100: reward=500.00 steps=500
  Ep 17/100: reward=500.00 steps=500
  Ep 18/100: reward=500.00 steps=500
  Ep 19/100: reward=500.00 steps=500
  Ep 20/100: reward=500.00 steps=500
  Ep 21/100: reward=500.00 steps=500
  Ep 22/100: reward=500.00 steps=500
  Ep 23/100: reward=500.00 steps=500
  Ep 24/100: reward=500.00 steps=500
  Ep 25/100: reward=500.00 steps=500
  Ep 26/100: reward=500.00 steps=500
  Ep 27/100: reward=500.00 steps=500
  Ep 28/10

d:\materials for college\Fall 2025-2026\Reinforcement Learning\assignments\assignment 2\.venv\Lib\site-packages\gymnasium\wrappers\rendering.py:293: UserWarning: WARN: Overwriting existing videos at d:\materials for college\Fall 2025-2026\Reinforcement Learning\assignments\assignment 2\videos\CartPole-v1\CartPole_DQN_batch-128 folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(
wandb: WARNING `fps` argument does not affect the frame rate of the video when providing a file path or raw bytes.


...


=== Model: CartPole_DQN_mem-50000.pth ===
W&B run: test_CartPole_DQN_mem-50000_1763011373


  Ep 1/100: reward=161.00 steps=161
  Ep 2/100: reward=166.00 steps=166
  Ep 3/100: reward=161.00 steps=161
  Ep 4/100: reward=163.00 steps=163
  Ep 5/100: reward=167.00 steps=167
  Ep 6/100: reward=165.00 steps=165
  Ep 7/100: reward=185.00 steps=185
  Ep 8/100: reward=172.00 steps=172
  Ep 9/100: reward=160.00 steps=160
  Ep 10/100: reward=162.00 steps=162
  Ep 11/100: reward=177.00 steps=177
  Ep 12/100: reward=184.00 steps=184
  Ep 13/100: reward=163.00 steps=163
  Ep 14/100: reward=179.00 steps=179
  Ep 15/100: reward=175.00 steps=175
  Ep 16/100: reward=175.00 steps=175
  Ep 17/100: reward=167.00 steps=167
  Ep 18/100: reward=179.00 steps=179
  Ep 19/100: reward=167.00 steps=167
  Ep 20/100: reward=182.00 steps=182
  Ep 21/100: reward=160.00 steps=160
  Ep 22/100: reward=164.00 steps=164
  Ep 23/100: reward=181.00 steps=181
  Ep 24/100: reward=165.00 steps=165
  Ep 25/100: reward=180.00 steps=180
  Ep 26/100: reward=180.00 steps=180
  Ep 27/100: reward=173.00 steps=173
  Ep 28/10

d:\materials for college\Fall 2025-2026\Reinforcement Learning\assignments\assignment 2\.venv\Lib\site-packages\gymnasium\wrappers\rendering.py:293: UserWarning: WARN: Overwriting existing videos at d:\materials for college\Fall 2025-2026\Reinforcement Learning\assignments\assignment 2\videos\CartPole-v1\CartPole_DQN_mem-50000 folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(
wandb: WARNING `fps` argument does not affect the frame rate of the video when providing a file path or raw bytes.
wandb: ERROR The nbformat package was not found. It is required to save notebook history.


Recorded video: videos/CartPole-v1/CartPole_DQN_mem-50000\CartPole_DQN_mem-50000-episode-0.mp4
Recording complete. Total reward: 174.00, Steps:174, Duration: 0.69 seconds


avg_reward,▁
best_reward,▁
episode,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇██
episode_reward,▃▃▅▂▆▆▃█▁▂▇▅▄▅▂▅▁▂▁▂▂▁▅▅▆▁▇▂▃▅▁█▆▃▃▄▅▁▅▅
recorded_episode_duration_sec,▁
recorded_episode_index,▁
recorded_episode_reward,▁
recorded_episode_steps,▁
steps,▃▂▃▃█▂▆█▆▅▃▇▃▄▅▄▃▂▂▃▃▂▄▅▂▃▁▁▅▃▂▂▄▂▅▇▄▂▄▁
worst_reward,▁
avg_reward,169.04


In [22]:
test_models_folder_with_wandb(
    env_name="Pendulum-v1",
    num_discrete_actions=5,
    models_folder="./pendulum Models",
    num_tests=100,
    wandb_project="RL A2 tests"
)


=== Model: Pendulum_DQN_eps-1000.pth ===
W&B run: test_Pendulum_DQN_eps-1000_1763011379


  Ep 1/100: reward=-1.60 steps=200
  Ep 2/100: reward=-312.99 steps=200
  Ep 3/100: reward=-0.76 steps=200
  Ep 4/100: reward=-222.63 steps=200
  Ep 5/100: reward=-2.53 steps=200
  Ep 6/100: reward=-345.55 steps=200
  Ep 7/100: reward=-126.08 steps=200
  Ep 8/100: reward=-126.07 steps=200
  Ep 9/100: reward=-123.12 steps=200
  Ep 10/100: reward=-125.10 steps=200
  Ep 11/100: reward=-120.84 steps=200
  Ep 12/100: reward=-236.27 steps=200
  Ep 13/100: reward=-126.25 steps=200
  Ep 14/100: reward=-237.72 steps=200
  Ep 15/100: reward=-118.20 steps=200
  Ep 16/100: reward=-124.75 steps=200
  Ep 17/100: reward=-0.91 steps=200
  Ep 18/100: reward=-118.84 steps=200
  Ep 19/100: reward=-117.49 steps=200
  Ep 20/100: reward=-231.51 steps=200
  Ep 21/100: reward=-227.53 steps=200
  Ep 22/100: reward=-3.25 steps=200
  Ep 23/100: reward=-0.93 steps=200
  Ep 24/100: reward=-228.46 steps=200
  Ep 25/100: reward=-124.38 steps=200
  Ep 26/100: reward=-4.55 steps=200
  Ep 27/100: reward=-120.06 steps=2

d:\materials for college\Fall 2025-2026\Reinforcement Learning\assignments\assignment 2\.venv\Lib\site-packages\gymnasium\wrappers\rendering.py:293: UserWarning: WARN: Overwriting existing videos at d:\materials for college\Fall 2025-2026\Reinforcement Learning\assignments\assignment 2\videos\Pendulum-v1\Pendulum_DQN_eps-1000 folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(
wandb: WARNING `fps` argument does not affect the frame rate of the video when providing a file path or raw bytes.
wandb: ERROR The nbformat package was not found. It is required to save notebook history.


Recorded video: videos/Pendulum-v1/Pendulum_DQN_eps-1000\Pendulum_DQN_eps-1000-episode-0.mp4
Recording complete. Total reward: -122.07, Steps:200, Duration: 1.03 seconds


avg_reward,▁
best_reward,▁
episode,▁▁▂▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▅▆▆▆▇▇▇▇▇▇███
episode_reward,█▂▄▁▆▃▆▃██▆▅▆██▃▆▆▆█▃▆▁▆▆▅▁▅▂▆▆▁█▆▁▆▃▆█▆
recorded_episode_duration_sec,▁
recorded_episode_index,▁
recorded_episode_reward,▁
recorded_episode_steps,▁
steps,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
worst_reward,▁
avg_reward,-131.50932


...


=== Model: Pendulum_DQN_mem-50000.pth ===
W&B run: test_Pendulum_DQN_mem-50000_1763011458


  Ep 1/100: reward=-298.96 steps=200
  Ep 2/100: reward=-243.20 steps=200
  Ep 3/100: reward=-117.87 steps=200
  Ep 4/100: reward=-113.90 steps=200
  Ep 5/100: reward=-126.69 steps=200
  Ep 6/100: reward=-115.57 steps=200
  Ep 7/100: reward=-123.60 steps=200
  Ep 8/100: reward=-121.27 steps=200
  Ep 9/100: reward=-123.31 steps=200
  Ep 10/100: reward=-255.99 steps=200
  Ep 11/100: reward=-2.12 steps=200
  Ep 12/100: reward=-0.99 steps=200
  Ep 13/100: reward=-124.85 steps=200
  Ep 14/100: reward=-121.44 steps=200
  Ep 15/100: reward=-118.77 steps=200
  Ep 16/100: reward=-233.24 steps=200
  Ep 17/100: reward=-0.69 steps=200
  Ep 18/100: reward=-123.56 steps=200
  Ep 19/100: reward=-122.59 steps=200
  Ep 20/100: reward=-122.39 steps=200
  Ep 21/100: reward=-124.96 steps=200
  Ep 22/100: reward=-113.67 steps=200
  Ep 23/100: reward=-123.48 steps=200
  Ep 24/100: reward=-122.48 steps=200
  Ep 25/100: reward=-120.82 steps=200
  Ep 26/100: reward=-124.67 steps=200
  Ep 27/100: reward=-121.37

d:\materials for college\Fall 2025-2026\Reinforcement Learning\assignments\assignment 2\.venv\Lib\site-packages\gymnasium\wrappers\rendering.py:293: UserWarning: WARN: Overwriting existing videos at d:\materials for college\Fall 2025-2026\Reinforcement Learning\assignments\assignment 2\videos\Pendulum-v1\Pendulum_DQN_mem-50000 folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(
wandb: WARNING `fps` argument does not affect the frame rate of the video when providing a file path or raw bytes.
wandb: ERROR The nbformat package was not found. It is required to save notebook history.


Recorded video: videos/Pendulum-v1/Pendulum_DQN_mem-50000\Pendulum_DQN_mem-50000-episode-0.mp4
Recording complete. Total reward: -119.04, Steps:200, Duration: 0.98 seconds


avg_reward,▁
best_reward,▁
episode,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▄▄▅▅▅▆▆▆▆▆▇▇▇▇▇███
episode_reward,▁▅▅█▅▅▅▅▅▅▃▅▂▅▅▃▅▂▃▅▅█▅▅█▃▅▅▅█▅▅▅▂▅▅▅▅█▅
recorded_episode_duration_sec,▁
recorded_episode_index,▁
recorded_episode_reward,▁
recorded_episode_steps,▁
steps,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
worst_reward,▁
avg_reward,-139.41974


In [16]:
test_models_folder_with_wandb(
    env_name="Acrobot-v1",
    num_discrete_actions=5,
    models_folder="./Acrobot models",
    num_tests=100,
    wandb_project="RL A2 tests"
)


=== Model: Acrobot_DQN_batch-128.pth ===
W&B run: test_Acrobot_DQN_batch-128_1763008229


wandb: ERROR The nbformat package was not found. It is required to save notebook history.


avg_reward,▁
best_reward,▁
episode,▁▁▂▂▂▂▂▂▃▃▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▇▇▇▇▇▇████
episode_reward,▅▇▄▇▅▄█▇▇▅▇▇▆▇▇▆▇▇▇▆▇▇▇▇▇▄▅▆▇▇▇▁▆▄▄▄██▇█
steps,█▂▅▃▅▂▂▂▂█▂▂▄▂▄▂▂▅▁▂▁▃▂▂▄▅▄▃▂▄▂▃▅▂▅▂▂▄▂▂
worst_reward,▁
avg_reward,-78.77
best_reward,-62
episode,99
episode_reward,-63
steps,64


  Ep 1/100: reward=-95.00 steps=96
  Ep 2/100: reward=-97.00 steps=98
  Ep 3/100: reward=-83.00 steps=84
  Ep 4/100: reward=-107.00 steps=108
  Ep 5/100: reward=-129.00 steps=130
  Ep 6/100: reward=-106.00 steps=107
  Ep 7/100: reward=-107.00 steps=108
  Ep 8/100: reward=-107.00 steps=108
  Ep 9/100: reward=-97.00 steps=98
  Ep 10/100: reward=-106.00 steps=107
  Ep 11/100: reward=-160.00 steps=161
  Ep 12/100: reward=-110.00 steps=111
  Ep 13/100: reward=-92.00 steps=93
  Ep 14/100: reward=-104.00 steps=105
  Ep 15/100: reward=-127.00 steps=128
  Ep 16/100: reward=-83.00 steps=84
  Ep 17/100: reward=-127.00 steps=128
  Ep 18/100: reward=-131.00 steps=132
  Ep 19/100: reward=-109.00 steps=110
  Ep 20/100: reward=-238.00 steps=239
  Ep 21/100: reward=-78.00 steps=79
  Ep 22/100: reward=-105.00 steps=106
  Ep 23/100: reward=-90.00 steps=91
  Ep 24/100: reward=-86.00 steps=87
  Ep 25/100: reward=-157.00 steps=158
  Ep 26/100: reward=-87.00 steps=88
  Ep 27/100: reward=-107.00 steps=108
  E

d:\materials for college\Fall 2025-2026\Reinforcement Learning\assignments\assignment 2\.venv\Lib\site-packages\gymnasium\wrappers\rendering.py:293: UserWarning: WARN: Overwriting existing videos at d:\materials for college\Fall 2025-2026\Reinforcement Learning\assignments\assignment 2\videos\Acrobot-v1\Acrobot_DQN_batch-128 folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(
wandb: WARNING `fps` argument does not affect the frame rate of the video when providing a file path or raw bytes.


...


=== Model: Acrobot_DQN_mem-50000.pth ===
W&B run: test_Acrobot_DQN_mem-50000_1763008352


  Ep 1/100: reward=-71.00 steps=72
  Ep 2/100: reward=-72.00 steps=73
  Ep 3/100: reward=-91.00 steps=92
  Ep 4/100: reward=-69.00 steps=70
  Ep 5/100: reward=-71.00 steps=72
  Ep 6/100: reward=-69.00 steps=70
  Ep 7/100: reward=-107.00 steps=108
  Ep 8/100: reward=-70.00 steps=71
  Ep 9/100: reward=-71.00 steps=72
  Ep 10/100: reward=-131.00 steps=132
  Ep 11/100: reward=-83.00 steps=84
  Ep 12/100: reward=-140.00 steps=141
  Ep 13/100: reward=-74.00 steps=75
  Ep 14/100: reward=-91.00 steps=92
  Ep 15/100: reward=-71.00 steps=72
  Ep 16/100: reward=-94.00 steps=95
  Ep 17/100: reward=-62.00 steps=63
  Ep 18/100: reward=-80.00 steps=81
  Ep 19/100: reward=-89.00 steps=90
  Ep 20/100: reward=-109.00 steps=110
  Ep 21/100: reward=-126.00 steps=127
  Ep 22/100: reward=-74.00 steps=75
  Ep 23/100: reward=-63.00 steps=64
  Ep 24/100: reward=-93.00 steps=94
  Ep 25/100: reward=-79.00 steps=80
  Ep 26/100: reward=-112.00 steps=113
  Ep 27/100: reward=-124.00 steps=125
  Ep 28/100: reward=-70

d:\materials for college\Fall 2025-2026\Reinforcement Learning\assignments\assignment 2\.venv\Lib\site-packages\gymnasium\wrappers\rendering.py:293: UserWarning: WARN: Overwriting existing videos at d:\materials for college\Fall 2025-2026\Reinforcement Learning\assignments\assignment 2\videos\Acrobot-v1\Acrobot_DQN_mem-50000 folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(
wandb: WARNING `fps` argument does not affect the frame rate of the video when providing a file path or raw bytes.
wandb: ERROR The nbformat package was not found. It is required to save notebook history.


Recorded video: videos/Acrobot-v1/Acrobot_DQN_mem-50000\Acrobot_DQN_mem-50000-episode-0.mp4
Recording complete. Total reward: -71.00, Steps:72, Duration: 0.44 seconds


avg_reward,▁
best_reward,▁
episode,▁▁▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇████
episode_reward,▅▇▇▇▁▆▄▇█▅▄▇▅▇▅▇▇▇▇▆▇▇▇▇▇▇▆▄▆█▇█▇▇▇▇▅▇▇▅
recorded_episode_duration_sec,▁
recorded_episode_index,▁
recorded_episode_reward,▁
recorded_episode_steps,▁
steps,▂▂▂▅▂▄▄▆▂▂▄▂▁▅▂▃▂▂▃▃▂▂▂▂▂▃▃▂▂▁▃▃▂▃▄▃▄█▂▄
worst_reward,▁
avg_reward,-81.27


In [18]:
test_models_folder_with_wandb(
    env_name="MountainCar-v0",
    num_discrete_actions=5,
    models_folder="./MountainCar models",
    num_tests=100,
    wandb_project="RL A2 tests"
)


=== Model: MountainCar_DQN_batch-128.pth ===
W&B run: test_MountainCar_DQN_batch-128_1763010684


wandb: ERROR The nbformat package was not found. It is required to save notebook history.


avg_reward,▁
best_reward,▁
episode,▁▁▁▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇████
episode_reward,██▁████▁▁▇██▁███████▇██▇▇▇▇▁▇▇▁▁▇▇█▁▁██▆
steps,▁▁█▁▁▁▁█▁█▁▁▁▁▁▂▁█▁█▂▁▁▂▁▂▁▁▁▁█▁▂██▁███▁
worst_reward,▁
avg_reward,-158.97
best_reward,-140
episode,99
episode_reward,-153
steps,153


  Ep 1/100: reward=-200.00 steps=200
  Ep 2/100: reward=-200.00 steps=200
  Ep 3/100: reward=-200.00 steps=200
  Ep 4/100: reward=-200.00 steps=200
  Ep 5/100: reward=-200.00 steps=200
  Ep 6/100: reward=-200.00 steps=200
  Ep 7/100: reward=-200.00 steps=200
  Ep 8/100: reward=-200.00 steps=200
  Ep 9/100: reward=-200.00 steps=200
  Ep 10/100: reward=-200.00 steps=200
  Ep 11/100: reward=-200.00 steps=200
  Ep 12/100: reward=-200.00 steps=200
  Ep 13/100: reward=-200.00 steps=200
  Ep 14/100: reward=-200.00 steps=200
  Ep 15/100: reward=-200.00 steps=200
  Ep 16/100: reward=-200.00 steps=200
  Ep 17/100: reward=-200.00 steps=200
  Ep 18/100: reward=-200.00 steps=200
  Ep 19/100: reward=-200.00 steps=200
  Ep 20/100: reward=-200.00 steps=200
  Ep 21/100: reward=-200.00 steps=200
  Ep 22/100: reward=-200.00 steps=200
  Ep 23/100: reward=-200.00 steps=200
  Ep 24/100: reward=-200.00 steps=200
  Ep 25/100: reward=-200.00 steps=200
  Ep 26/100: reward=-200.00 steps=200
  Ep 27/100: reward=-

d:\materials for college\Fall 2025-2026\Reinforcement Learning\assignments\assignment 2\.venv\Lib\site-packages\gymnasium\wrappers\rendering.py:293: UserWarning: WARN: Overwriting existing videos at d:\materials for college\Fall 2025-2026\Reinforcement Learning\assignments\assignment 2\videos\MountainCar-v0\MountainCar_DQN_batch-128 folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(
wandb: WARNING `fps` argument does not affect the frame rate of the video when providing a file path or raw bytes.


...

avg_reward,▁
best_reward,▁
episode,▁▁▁▂▂▃▃▃▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇██
episode_reward,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
recorded_episode_duration_sec,▁
recorded_episode_index,▁
recorded_episode_reward,▁
recorded_episode_steps,▁
steps,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
worst_reward,▁
avg_reward,-200



=== Model: MountainCar_DQN_mem-10000 .pth ===
W&B run: test_MountainCar_DQN_mem-10000 _1763010782


  Ep 1/100: reward=-140.00 steps=140
  Ep 2/100: reward=-143.00 steps=143
  Ep 3/100: reward=-143.00 steps=143
  Ep 4/100: reward=-200.00 steps=200
  Ep 5/100: reward=-151.00 steps=151
  Ep 6/100: reward=-145.00 steps=145
  Ep 7/100: reward=-143.00 steps=143
  Ep 8/100: reward=-200.00 steps=200
  Ep 9/100: reward=-144.00 steps=144
  Ep 10/100: reward=-200.00 steps=200
  Ep 11/100: reward=-142.00 steps=142
  Ep 12/100: reward=-141.00 steps=141
  Ep 13/100: reward=-145.00 steps=145
  Ep 14/100: reward=-140.00 steps=140
  Ep 15/100: reward=-144.00 steps=144
  Ep 16/100: reward=-140.00 steps=140
  Ep 17/100: reward=-144.00 steps=144
  Ep 18/100: reward=-151.00 steps=151
  Ep 19/100: reward=-140.00 steps=140
  Ep 20/100: reward=-140.00 steps=140
  Ep 21/100: reward=-199.00 steps=199
  Ep 22/100: reward=-140.00 steps=140
  Ep 23/100: reward=-147.00 steps=147
  Ep 24/100: reward=-149.00 steps=149
  Ep 25/100: reward=-144.00 steps=144
  Ep 26/100: reward=-200.00 steps=200
  Ep 27/100: reward=-

d:\materials for college\Fall 2025-2026\Reinforcement Learning\assignments\assignment 2\.venv\Lib\site-packages\gymnasium\wrappers\rendering.py:293: UserWarning: WARN: Overwriting existing videos at d:\materials for college\Fall 2025-2026\Reinforcement Learning\assignments\assignment 2\videos\MountainCar-v0\MountainCar_DQN_mem-10000 folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'videos/MountainCar-v0/MountainCar_DQN_mem-10000 '